In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
import lxml
from datetime import datetime
from selenium.webdriver.common.keys import Keys

In [6]:

search_box_text = "sports shoes for women"
website_link = "https://www.flipkart.com/"

session_start_time = datetime.now().time()
print(f"Session Start Time: {session_start_time} -------------------------->")

driver = webdriver.Chrome()
driver.get(website_link)
driver.maximize_window()
wait = WebDriverWait(driver, 30)

# ---------- Close login popup if present ----------
try:
    close_btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button._2KpZ6l._2doB4z"))
    )
    close_btn.click()
    print("Popup closed")
except:
    print("No popup appeared")

# ---------- Search ----------
print("Waiting for search box...")
search_input = wait.until(
    EC.presence_of_element_located((By.NAME, "q"))
)

print("Typing search query...")
search_input.send_keys(search_box_text)
search_input.send_keys(Keys.RETURN)

# ---------- Wait for results ----------
wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="/p/"]'))
)

print("Collecting product links...")

all_product_links = []

while True:
    time.sleep(2)

    products = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/p/"]')
    links = [p.get_attribute("href") for p in products if p.get_attribute("href")]
    all_product_links.extend(links)

    print(f"Collected {len(all_product_links)} links so far")

    # ---------- Try clicking Next ----------
    try:
        next_btn = driver.find_element(By.XPATH, '//a/span[text()="Next"]/..')

        driver.execute_script("arguments[0].scrollIntoView();", next_btn)
        time.sleep(1)

        driver.execute_script("arguments[0].click();", next_btn)

        wait.until(
            EC.staleness_of(products[0])
        )

    except:
        print("No more pages")
        break

# ---------- Remove duplicates ----------
all_product_links = list(set(all_product_links))

print("Total Unique Product Links:", len(all_product_links))

# ---------- Save CSV ----------
df = pd.DataFrame(all_product_links, columns=["product_links"])
df.to_csv("flipkart_product_links.csv", index=False)

driver.quit()

session_end_time = datetime.now().time()
print(f"Session End Time: {session_end_time} -------------------------->")

Session Start Time: 15:13:50.639314 -------------------------->
No popup appeared
Waiting for search box...
Typing search query...
Collected 125 links so far
Collected 250 links so far
Collected 375 links so far
Collected 500 links so far
Collected 625 links so far
Collected 750 links so far
Collected 875 links so far
Collected 1000 links so far
Collected 1125 links so far
Collected 1250 links so far
Collected 1375 links so far
Collected 1500 links so far
Collected 1625 links so far
Collected 1750 links so far
Collected 1875 links so far
Collected 2000 links so far
Collected 2125 links so far
Collected 2250 links so far
Collected 2375 links so far
Collected 2500 links so far
Collected 2625 links so far
Collected 2750 links so far
Collected 2875 links so far
Collected 3000 links so far
Collected 3125 links so far
Collected 3250 links so far
Collected 3375 links so far
Collected 3500 links so far
Collected 3625 links so far
Collected 3750 links so far
Collected 3875 links so far
Collecte

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pandas as pd
import time
import re

# session start time
session_start_time = datetime.now().time()
print(f"Session Start Time: {session_start_time} ---------------------------> ")

# read csv of product links
df_product_links = pd.read_csv("flipkart_product_links.csv")

# remove limit if scraping all
df_product_links = df_product_links.head(10)

all_product_links = df_product_links['product_links'].tolist()
print("Collecting Individual Product Detail Information")

# start browser
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)

complete_product_details = []
unavailable_products = []

success = 0
failed = 0

for product_page_link in all_product_links:
    try:
        driver.get(product_page_link)

        # ---------- close popup if appears ----------
        try:
            close_btn = WebDriverWait(driver,5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR,'button._2KpZ6l._2doB4z'))
            )
            close_btn.click()
        except:
            pass

        # ---------- wait for page ----------
        wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
        wait.until(EC.presence_of_element_located((By.TAG_NAME,"body")))

        # ---------- check unavailable ----------
        try:
            status = driver.find_element(By.XPATH,"//*[contains(text(),'Unavailable') or contains(text(),'Sold Out')]").text
            unavailable_products.append(product_page_link)
            print("Unavailable:", product_page_link)
            continue
        except:
            pass

        # ---------- brand ----------
        try:
            brand = driver.find_element(By.CSS_SELECTOR,"span.mEh187").text
        except:
            brand = ""

        # ---------- title ----------
        try:
            title = driver.find_element(By.CSS_SELECTOR,"span.VU-ZEz").text
            title = re.sub(r'\s*\([^)]*\)', '', title)
        except:
            title = ""

        # ---------- price ----------
        try:
            price = driver.find_element(By.CSS_SELECTOR,"div.Nx9bqj").text
            price = int(re.sub(r"[^\d]","",price))
        except:
            price = ""

        # ---------- discount ----------
        try:
            discount = driver.find_element(By.CSS_SELECTOR,"div.UkUFwK").text
            discount = int(re.sub(r"[^\d]","",discount))
        except:
            discount = ""

        # ---------- ratings ----------
        try:
            avg_rating = driver.find_element(By.CSS_SELECTOR,"div.XQDdHH").text
        except:
            avg_rating = ""

        try:
            total_ratings = driver.find_element(By.CSS_SELECTOR,"span.Wphh3N").text
            total_ratings = int(re.sub(r"[^\d]","", total_ratings))
        except:
            total_ratings = ""

        # ---------- save ----------
        complete_product_details.append([
            product_page_link,
            title,
            brand,
            price,
            discount,
            avg_rating,
            total_ratings
        ])

        success += 1
        print(f"Success {success}")

    except Exception as e:
        failed += 1
        unavailable_products.append(product_page_link)
        print("Failed:", product_page_link, e)

driver.quit()

# ---------- dataframe ----------
df = pd.DataFrame(
    complete_product_details,
    columns=['product_link','title','brand','price','discount','avg_rating','total_ratings']
)

# duplicates
df_duplicate_products = df[df.duplicated()]
df = df.drop_duplicates()

# unavailable df
df_unavailable_products = pd.DataFrame(unavailable_products, columns=['link'])

# ---------- stats ----------
print("Total product pages:", len(all_product_links))
print("Final products:", len(df))
print("Unavailable:", len(df_unavailable_products))
print("Duplicates:", len(df_duplicate_products))

# ---------- save ----------
df.to_csv("flipkart_product_data.csv", index=False)
df_unavailable_products.to_csv("unavailable_products.csv", index=False)
df_duplicate_products.to_csv("duplicate_products.csv", index=False)

session_end_time = datetime.now().time()
print(f"Session End Time: {session_end_time} ---------------------------> ")

Session Start Time: 15:19:40.258937 ---------------------------> 
Success 1
Success 2
Success 3
Success 4
Success 5
Success 6
Success 7
Success 8
Success 9
Success 10
Total product pages: 10
Final products: 10
Unavailable: 0
Duplicates: 0
Session End Time: 15:21:10.693596 ---------------------------> 
